In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import os

In [2]:
from keras import models
from keras import layers
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [3]:
Imagesize=64
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(Imagesize,Imagesize,3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D(2, 2))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D(2, 2))
model.add(layers.Conv2D(256, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D(2, 2))
model.add(layers.Flatten())
model.add(layers.Dropout(0.25))
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(25, activation='softmax'))

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=1, min_lr=0.0001)
early_stop = EarlyStopping(monitor='val_loss', min_delta=0, patience=2, verbose=0, mode='auto')

In [ ]:
model.summary()

In [ ]:
data_train = ImageDataGenerator(rescale=1.0/255.0,rotation_range=40,width_shift_range=0.2,height_shift_range=0.2,
                                shear_range=0.2,zoom_range=0.2,horizontal_flip=True)
data_valid = ImageDataGenerator(rescale=1.0/255.0)

In [ ]:
batch = 96

In [ ]:
train_gen = data_train.flow_from_directory('train_processed',target_size=(Imagesize,Imagesize),batch_size=batch, 
                                           class_mode='categorical')

test_gen = data_valid.flow_from_directory('test_processed', target_size=(Imagesize,Imagesize), batch_size=batch, 
                                           class_mode='categorical')

In [ ]:
model.fit_generator(train_gen, epochs=3, steps_per_epoch=50, validation_data = test_gen, 
                    validation_steps=4, callbacks=[reduce_lr, early_stop])

In [ ]:
#to create a saved model, use the following command
#model.save('islcnnmodel.h5')

In [ ]:
imgs,labels=next(test_gen)
scores = model.evaluate(imgs,labels,verbose=0)
print("Accuracy: ",scores[1])